In [1]:
import pandas as pd
import dtale as dtale
import warnings
warnings.filterwarnings('ignore')

In [2]:
#https://hdr.undp.org/sites/default/files/2023-24_HDR/HDR23-24_Composite_indices_complete_time_series.csv
df_hd = pd.read_csv('../Trabalho UX/HDI.csv', sep=',', encoding='ANSI',low_memory=False)

In [3]:
#vendo as colunas
df_hd.columns

Index(['iso3', 'country', 'hdicode', 'region', 'hdi_rank_2022', 'hdi_1990',
       'hdi_1991', 'hdi_1992', 'hdi_1993', 'hdi_1994',
       ...
       'pop_total_2013', 'pop_total_2014', 'pop_total_2015', 'pop_total_2016',
       'pop_total_2017', 'pop_total_2018', 'pop_total_2019', 'pop_total_2020',
       'pop_total_2021', 'pop_total_2022'],
      dtype='object', length=1076)

In [4]:
# Colunas a manter
id_vars = ['country']

In [5]:
# removendo colunas desnecessárias
df_hd = df_hd.drop(columns=[col for col in df_hd.columns if 'iso3' in col])
df_hd = df_hd.drop(columns=[col for col in df_hd.columns if 'region' in col])
df_hd = df_hd.drop(columns=[col for col in df_hd.columns if 'hdicode' in col])

In [6]:
# Colunas a transformar
value_vars = [col for col in df_hd.columns if col not in id_vars]

In [7]:
# melt para transformar os indicadores em uma única coluna
df_melted = pd.melt(df_hd, id_vars=id_vars, value_vars=value_vars, var_name='indicator_year', value_name='value')

In [8]:
#dtale.show(df_melted)

In [9]:
# Separando os indicadores dos anos
df_melted[['indicator', 'year']] = df_melted['indicator_year'].str.extract(r'([a-zA-Z_]+)(\d{4})')

In [10]:
# Verificar se algum país deixou de existir após o pivotamento
print("Antes do melt: ", df_hd['country'].nunique())
print("Depois do melt: ", df_melted['country'].nunique())

Antes do melt:  206
Depois do melt:  206


In [11]:
# Filtrando apenas os anos entre 2017 e 2021
df_filtered = df_melted[df_melted['year'].astype(int).between(2017, 2021)]

In [12]:
# conferencia se algum país deixou de existir após o pivotamento
print("Antes do ano: ", df_hd['country'].nunique())
print("Depois do ano: ", df_melted['country'].nunique())

Antes do ano:  206
Depois do ano:  206


In [13]:
# Remover o sufixo adicional "_" nas colunas de indicadores, se houver
df_filtered['indicator'] = df_filtered['indicator'].str.rstrip('_')

In [14]:
#dtale.show(df_filtered)

In [15]:
# Pivotando os dados
df_pivoted = df_filtered.pivot_table(index=id_vars + ['year'], columns='indicator', values='value', aggfunc='first').reset_index()

In [16]:
#dtale.show(df_pivoted)

In [17]:
# Verificar se algum país deixou de existir após o pivotamento
print("Antes do pivotamento: ", df_filtered['country'].nunique())
print("Depois do pivotamento: ", df_pivoted['country'].nunique())

Antes do pivotamento:  206
Depois do pivotamento:  206


In [18]:
# Renomear as colunas conforme os metadados:
# https://hdr.undp.org/sites/default/files/2023-24_HDR/HDR23-24_Composite_indices_metadata.xlsx
rename_dict = {
    'abr': 'Adolescent Birth Rate (births per 1,000 women ages 15-19)',
    'co2_prod': 'Carbon dioxide emissions per capita (production) (tonnes)',
    'coef_ineq': 'Coefficient of human inequality',
    'country': 'HDR Country Name',
    'diff_hdi_phdi': 'Difference from HDI value (%)',
    'eys': 'Expected Years of Schooling (years)',
    'eys_f': 'Expected Years of Schooling, female (years)',
    'eys_m': 'Expected Years of Schooling, male (years)',
    'gdi': 'Gender Development Index (value)',
    'gdi_group': 'GDI Group',
    'gii': 'Gender Inequality Index (value)',
    'gii_rank': 'GII Rank',
    'gni_pc_f': 'Gross National Income Per Capita, female (2017 PPP$)',
    'gni_pc_m': 'Gross National Income Per Capita, male (2017 PPP$)',
    'gnipc': 'Gross National Income Per Capita (2017 PPP$)',
    'hdi': 'Human Development Index (value)',
    'hdi_f': 'HDI female',
    'hdi_m': 'HDI male',
    'hdicode': 'Human Development Groups',
    'ihdi': 'Inequality-adjusted Human Development Index (value)',
    'ineq_edu': 'Inequality in eduation',
    'ineq_inc': 'Inequality in income',
    'ineq_le': 'Inequality in life expectancy',
    'iso3': 'ISO3',
    'le': 'Life Expectancy at Birth (years)',
    'le_f': 'Life Expectancy at Birth, female (years)',
    'le_m': 'Life Expectancy at Birth, male (years)',
    'lfpr_f': 'Labour force participation rate, female (% ages 15 and older)',
    'lfpr_m': 'Labour force participation rate, male (% ages 15 and older)',
    'loss': 'Overall loss (%)',
    'mf': 'Material footprint per capita (tonnes)',
    'mmr': 'Maternal Mortality Ratio (deaths per 100,000 live births)',
    'mys': 'Mean Years of Schooling (years)',
    'mys_f': 'Mean Years of Schooling, female (years)',
    'mys_m': 'Mean Years of Schooling, male (years)',
    'phdi': 'Planetary pressures–adjusted Human Development Index (value)',
    'pop_total': 'Population, total (millions)',
    'pr_f': 'Share of seats in parliament, female (% held by women)',
    'pr_m': 'Share of seats in parliament, male (% held by men)',
    'rankdiff_hdi_phdi': 'Difference from HDI rank',
    'region': 'UNDP Developeing Regions',
    'se_f': 'Population with at least some secondary education, female (% ages 25 and older)',
    'se_m': 'Population with at least some secondary education, male (% ages 25 and older)'
}
df_pivoted.rename(columns=rename_dict, inplace=True)

In [19]:
#Drop da coluna nao interessante para estudo
df_pivoted = df_pivoted.drop(columns=[col for col in df_pivoted.columns if '_prod' in col])
#dtale.show(df_pivoted)

In [20]:
#Remover aglomerado de paises
remover = [
    'Arab States',
    'East Asia and the Pacific',
    'Europe and Central Asia',
    'High human development',
    'Hong Kong, China (SAR)',
    'Latin America and the Caribbean',
    'Low human development',
    'Medium human development',
    'South Asia',
    'Sub-Saharan Africa',
    'Liechtenstein',
    'Monaco',
    'San Marino',
    'Very high human development',
    'World'
]
df_pivoted = df_pivoted[~df_pivoted['HDR Country Name'].isin(remover)]

In [21]:
# Verificar se algum país deixou de existir após o pivotamento
print("Antes do pivot: ", df_filtered['country'].nunique())
print("Depois do pivot: ", df_pivoted['HDR Country Name'].nunique())

Antes do pivot:  206
Depois do pivot:  191


In [22]:
#Verificando se existem dados nulos
print(df_pivoted.isnull().sum())

indicator
HDR Country Name                                                                     0
year                                                                                 0
Adolescent Birth Rate (births per 1,000 women ages 15-19)                            0
Coefficient of human inequality                                                    153
Difference from HDI value (%)                                                      190
Expected Years of Schooling (years)                                                 10
Expected Years of Schooling, female (years)                                         10
Expected Years of Schooling, male (years)                                           10
Gender Development Index (value)                                                    71
Gender Inequality Index (value)                                                    138
Gross National Income Per Capita, female (2017 PPP$)                                68
Gross National Income Per Capita,

In [23]:
#existem dados nulos, apagando todas as linhas nulas
#nao apagar nulos neste momento, somente após o merge das bases
#df_pivoted.dropna(inplace=True)

In [24]:
df_codigo = pd.read_csv('../Trabalho UX/area_code.csv', sep=',', encoding='utf-8',low_memory=False)

In [25]:
#nao encontramos alguns paises, vamos ajustar o DF para que seja o mesmo nome do area code
rename_country = {
    'Netherlands': 'Netherlands (Kingdom of the)',
    'United States': 'United States of America',
    "Korea (Democratic People's Rep. of)": "Democratic People's Republic of Korea",
    'Korea (Republic of)': 'Republic of Korea',
    'Moldova (Republic of)': 'Republic of Moldova',
    'Congo (Democratic Republic of the)': 'Democratic Republic of the Congo',
    'Palestine, State of': 'Palestine',
    'Tanzania (United Republic of)': 'United Republic of Tanzania',
    'United Kingdom': 'United Kingdom of Great Britain and Northern Ireland',
    'Eswatini (Kingdom of)': 'Eswatini'
}

# Renomear os países no df_pivoted
df_pivoted['HDR Country Name'] = df_pivoted['HDR Country Name'].replace(rename_country)

In [26]:
#Colocando o código de area no DF
df_merge = pd.merge(df_pivoted, df_codigo, left_on='HDR Country Name', right_on='Area', how='left')
df_merge = df_merge.drop(columns=['Area'])

In [27]:
# Reordenando as colunas para que 'Area Code' seja a primeira
cols = ['Area Code'] + [col for col in df_merge.columns if col != 'Area Code']
df_merge = df_merge[cols]

In [28]:
# Renomear as colunas
df_merge = df_merge.rename(columns={
    'HDR Country Name': 'Area',
    'year': 'Year'
})

In [29]:
#dtale.show(df_merge)

In [30]:
#dtale.show(df_codigo)

In [31]:
# Salvando o DataFrame em um arquivo CSV
df_merge.to_csv('Emerson-HDI.csv', index=False)